In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# export

import time
import logging

from django.http import FileResponse

logger = logging.getLogger(__name__)

<IPython.core.display.Javascript object>

## Measuring File Response

In [ ]:
# export


class MeasuringFileResponse(FileResponse):
    def __init__(self, *args, **kwargs):
        self.open_file_name = args[0].name
        self.started_serving = time.perf_counter()
        super().__init__(*args, **kwargs)

    def close(self):
        super().close()
        elapsed = time.perf_counter() - self.started_serving
        # print(f"elapsed: {elapsed} for {self.open_file_name}")
        logger.info(f"{self.open_file_name},{elapsed}")

<IPython.core.display.Javascript object>

## Async File Response

In [ ]:
# default_exp django.http

<IPython.core.display.Javascript object>

In [ ]:
# export

import aiofiles


class AsyncFileResponse(FileResponse):
    def __init__(self, *args, **kwargs):
        self.started_serving = time.perf_counter()
        self.chunk_size = kwargs.get("chunk_size", 4096)
        self.async_file = None
        self.more_body = True
        super().__init__(*args, **kwargs)
        self.headers["Content-Type"] = "application/octet-stream"

    def __aiter__(self):
        return self

    async def __anext__(self):
        if self.async_file is None:
            self.async_file = await aiofiles.open(self.filename, mode="rb")
        while self.more_body:
            chunk = await self.async_file.read(self.chunk_size)
            self.more_body = len(chunk) == self.chunk_size
            return self.more_body, chunk
        await self.async_file.close()
        raise StopAsyncIteration

    def close(self):
        super().close()
        elapsed = time.perf_counter() - self.started_serving
        logger.info(f"{self.filename},{elapsed}")

<IPython.core.display.Javascript object>

In [ ]:
# dont_test
import os
import asyncio

import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "will_it_saturate.django.settings")

django.setup(set_prefix=False)


afr = AsyncFileResponse()
print(asyncio.iscoroutinefunction(afr.__aiter__))
print(hasattr(afr, "__aiter__"))

False
True


<IPython.core.display.Javascript object>

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_index.ipynb.
Converted 01_host.ipynb.
Converted 02_file.ipynb.
Converted 03_registry.ipynb.
Converted 04_epochs.ipynb.
Converted 10_servers.ipynb.
Converted 11_views_for_fastapi_server.ipynb.
Converted 12_views_for_django_server.ipynb.
Converted 13_django_asgi.ipynb.
Converted 14_django_async_file_response.ipynb.
Converted 15_servers_started_locally.ipynb.
Converted 16_servers_started_via_docker.ipynb.
Converted 20_clients.ipynb.
Converted 21_benchmark_client_implementations.ipynb.
Converted 30_control_server.ipynb.
Converted 31_control_client.ipynb.
Converted 32_control_cli.ipynb.
Converted 40_results.ipynb.
Converted 41_repositories.ipynb.
Converted 42_sqlite_repository.ipynb.
Converted 50_benchmark_without_benchmark.ipynb.
Converted 51_benchmark_remote_control_server.ipynb.
Converted 52_benchmark_from_config.ipynb.
Converted 52_generate_files_for_benchmark.ipynb.
Converted 53_analyze.ipynb.
Converted 60_legacy.ipynb.
Converted 61_run_benchmark.ipynb.


<IPython.core.display.Javascript object>